# ETL

## Extract - Transform - Load

En esta sección nos encargaremos de hacer las modificaciones para nuestros dataset, despues su importación para tener un dataset más limpio y fácil de manejar. Las transfomaciones serán en base a lo visto en la EDA.

In [34]:
#Importamos las librerias con las que vamos a trabajar solamente
import pandas as pd
import ast
import pyarrow as pa
import pyarrow.parquet as pq

### 1er DataSet: Steam Games

Como se pudo observar en el desarrollo de la EDA, primero nos vamos a centrar el dataset de steam games, en el cual solamente vamos a limpiar los valores vacios, posteriormente ordenar alfabeticamente y al final vamos a transformar el archivo a un .parquet para tener un archivo más ligero.

In [35]:
#Comenzamos con la lectura y limpieza de nulos
df_Raw_steamGames = pd.read_json(r'..\DataRaw\\output_steam_games.json', lines= True)

df_Raw_steamGames = df_Raw_steamGames.sort_values(by= 'publisher', ascending= True)

df_steamGames_clean = df_Raw_steamGames.dropna()

#Para una mejor limpieza de columans que no sean utiles para las funcioens
df_steamGames_cleanf = df_steamGames_clean
df_steamGames_cleanf = df_steamGames_cleanf.drop('url', axis= 1)
df_steamGames_cleanf = df_steamGames_cleanf.drop('reviews_url', axis= 1)
df_steamGames_cleanf = df_steamGames_cleanf.drop('app_name', axis= 1)

#Eliminamos la palabra Free to play de la columna de precios para tener solo valores flotantes
price = list(df_steamGames_cleanf['price'])

for i in range(len(price)):

    if type(price[i]) == str:
        price[i] = 0.0

df_steamGames_cleanf['price'] = price

In [36]:
#Ahora separamos el año en una nueva columna 
años_list = df_steamGames_cleanf['release_date']
años = []

for i in años_list:
    años.append(i.split('-')[0])

df_steamGames_cleanf['year'] = años


In [37]:
df_steamGames_cleanf.head()

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer,year
92098,(none),"[Indie, Simulation, Strategy, Early Access]",Stonehearth,2015-06-03,"[Early Access, City Builder, Sandbox, Building...","[Single-player, Steam Trading Cards]",24.99,1.0,253250.0,Radiant Entertainment,2015
106822,(none),[Action],Scribble Ships,2017-03-29,"[Action, Arcade, Indie]","[Single-player, Steam Achievements, Steam Trad...",3.99,0.0,548130.0,Slothzilla Games,2017
109573,+7 Software,"[Action, Casual, Indie, Early Access]",Final Days,2016-10-03,"[Early Access, Action, Indie, Casual, Co-op, Z...","[Single-player, Multi-player, Co-op, Online Co...",4.99,1.0,459830.0,+7 Software,2016
103398,"+Mpact Games, LLC.","[Action, Indie, Early Access]",Hanako: Honor &amp; Blade,2017-10-09,"[Early Access, Multiplayer, Action, Indie, Med...","[Multi-player, Online Multi-Player, Steam Achi...",9.99,1.0,349510.0,"+Mpact Games, LLC.",2017
90612,-,"[Action, Indie, Early Access]",XenoRaptor,2014-06-16,"[Early Access, Action, Indie, Bullet Hell, Spa...","[Single-player, Multi-player, Co-op, Full cont...",9.99,1.0,298280.0,Peter Cleary,2014


In [38]:
df_steamGames_cleanf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 92098 to 107892
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   title         22530 non-null  object 
 3   release_date  22530 non-null  object 
 4   tags          22530 non-null  object 
 5   specs         22530 non-null  object 
 6   price         22530 non-null  float64
 7   early_access  22530 non-null  float64
 8   id            22530 non-null  float64
 9   developer     22530 non-null  object 
 10  year          22530 non-null  object 
dtypes: float64(3), object(8)
memory usage: 2.1+ MB


Podemos observar que ya contamos con la misma cantidad de entradas y valores en cada columna
por lo tanto ya podemos trabajar solamente con datos completos. Ahora procedemos exportar nuestro
dataset a un archivo parquet para tener un archivo de menor peso y más eficiente en su lectura.

In [39]:
#Exportamos nuestros datos a un archivos parquet.
df_steamGames_cleanf.to_parquet(r'..\DataParquet\\SteamGames.parquet')

In [40]:
#Ahora solo leemos el archivo parquet para asegurarnos de que este funcionando sin problemas
df =pd.read_parquet(r'..\DataParquet\\SteamGames.parquet')
df.head()

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer,year
92098,(none),"[Indie, Simulation, Strategy, Early Access]",Stonehearth,2015-06-03,"[Early Access, City Builder, Sandbox, Building...","[Single-player, Steam Trading Cards]",24.99,1.0,253250.0,Radiant Entertainment,2015
106822,(none),[Action],Scribble Ships,2017-03-29,"[Action, Arcade, Indie]","[Single-player, Steam Achievements, Steam Trad...",3.99,0.0,548130.0,Slothzilla Games,2017
109573,+7 Software,"[Action, Casual, Indie, Early Access]",Final Days,2016-10-03,"[Early Access, Action, Indie, Casual, Co-op, Z...","[Single-player, Multi-player, Co-op, Online Co...",4.99,1.0,459830.0,+7 Software,2016
103398,"+Mpact Games, LLC.","[Action, Indie, Early Access]",Hanako: Honor &amp; Blade,2017-10-09,"[Early Access, Multiplayer, Action, Indie, Med...","[Multi-player, Online Multi-Player, Steam Achi...",9.99,1.0,349510.0,"+Mpact Games, LLC.",2017
90612,-,"[Action, Indie, Early Access]",XenoRaptor,2014-06-16,"[Early Access, Action, Indie, Bullet Hell, Spa...","[Single-player, Multi-player, Co-op, Full cont...",9.99,1.0,298280.0,Peter Cleary,2014


Podemos observar que logramos leer nuestro archivo parquet con los datos y columnas ya editados, así que ya tenemos nuestro archivo con el que vamos a trabajar para las funciones.

### 2do DataSet: Australian Items

Para este dataset lo más importante va a ser el expandir nuestro dataframe desanidando información de la columa de items y posteriormente ordenar, ya que como observamos en la EDA no existen datos vacios para limpiar. 

También eliminaremos las columnas que no sean importantes para las funciones que se van a desarrollar.

In [41]:
#Leemos el archivo linea por linea para poder decodificar
rows = []

with open(r'..\DataRaw\\australian_users_items.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

f.close()
    
df_australian_items = pd.DataFrame(rows)

In [42]:
df_australian_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [43]:
df_australian_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [44]:
df1_items = df_australian_items.drop('user_url', axis= 1)
df1_items = df1_items.drop('items', axis= 1)


In [45]:
df1_items.head()

,user_id,items_count,steam_id
0,76561197970982479,277,76561197970982479
1,js41637,888,76561198035864385
2,evcentric,137,76561198007712555
3,Riot-Punch,328,76561197963445855
4,doctr,541,76561198002099482


In [46]:
# Aplicar json_normalize directamente en la columna 'items' y conservar 'user_id':
df_expanded_items = pd.json_normalize(df_australian_items.to_dict('records'), 'items', ['user_id'])

# Reorganizar las columnas:
df2_expanded = df_expanded_items[['user_id', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']]

In [47]:
df2_expanded.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [48]:
df2_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   item_name         object
 3   playtime_forever  int64 
 4   playtime_2weeks   int64 
dtypes: int64(2), object(3)
memory usage: 196.6+ MB


En este caso para las reviews vamos a tener que tener dos parquets diferentes, ya que contienen diferente número de filas. Por lo tanto el primero va a ser el dataset sin la columna de reviews y el otro el expanded de las reviews. Así que procedemos a hacer esa importació a .parquet.

In [49]:
#Exportamos nuestros datos a un archivos parquet.
df1_items.to_parquet(r'..\DataParquet\\AustItems.parquet')
df2_expanded.to_parquet(r'..\DataParquet\\AustItemsExpand.parquet')

In [50]:
#Ahora solo leemos el archivo parquet para asegurarnos de que este funcionando sin problemas
df =pd.read_parquet(r'..\DataParquet\\AustItems.parquet')
df.head()

,user_id,items_count,steam_id
0,76561197970982479,277,76561197970982479
1,js41637,888,76561198035864385
2,evcentric,137,76561198007712555
3,Riot-Punch,328,76561197963445855
4,doctr,541,76561198002099482


In [51]:
#Lo mismo para el segundo archivo parquet, esto nuevamente para asegurarnos de que este funcionando sin problemas
df =pd.read_parquet(r'..\DataParquet\\AustItemsExpand.parquet')
df.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


### 3er DataSet: Australian Reviews

Para este al igual que en el segundo dataset, solo es necesario desanidar información de una de las columnas, ya que no cuentan con datos vacios.

In [52]:
#Tenemos el mismo problema que con el dataset pasado, así que utilizaremos la misma decodificación
rows = []

with open(r'..\DataRaw\\australian_user_reviews.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

f.close()
    
df_aust_reviews = pd.DataFrame(rows)

In [53]:
df_aust_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [54]:
df_aust_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [55]:
# Aplicar json_normalize directamente en la columna 'items' y conservar 'user_id':
df_expanded_reviews = pd.json_normalize(df_aust_reviews.to_dict('records'), 'reviews', ['user_id'])

# Reorganizar las columnas:
df_User_Reviews_clean = df_expanded_reviews[['user_id', 'item_id', 'helpful', 'recommend', 'review']]

df_reviews_expand = df_User_Reviews_clean.sort_values(by= 'user_id', ascending= True)

In [56]:
df_reviews_expand.head()

,user_id,item_id,helpful,recommend,review
21123,--000--,1250,1 of 1 people (100%) found this review helpful,True,‡∏´‡∏ô‡∏∏‡∏Å‡∏Ñ‡∏±‡∏ö ‡πÅ‡∏ô‡∏∞‡∏ô‡∏≥ 10/10
58209,--ace--,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :) ...
58208,--ace--,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed
32142,--ionex--,730,No ratings yet,True,"it done brokeded on me, the game no longer wor..."
32143,--ionex--,105600,No ratings yet,True,"It's an amazing game, and im glad that mac use..."


In [57]:
df_reviews_expand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 21123 to 42588
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  object
 2   helpful    59305 non-null  object
 3   recommend  59305 non-null  bool  
 4   review     59305 non-null  object
dtypes: bool(1), object(4)
memory usage: 2.3+ MB


Para este último dataset solo vamos a generar un archivo solamente, en este tenemos toda la información necesaria para poder hacer las consultas pertienentes en las funciones que se vana desarrollar en la siguiente parte del proyecto.

In [58]:
#Exportamos nuestros datos a un archivos parquet.
df_reviews_expand.to_parquet(r'..\DataParquet\\AustReviews.parquet')

In [59]:
#Ahora solo leemos el archivo parquet para asegurarnos de que este funcionando sin problemas
df =pd.read_parquet(r'..\DataParquet\\AustReviews.parquet')
df.head()

,user_id,item_id,helpful,recommend,review
21123,--000--,1250,1 of 1 people (100%) found this review helpful,True,‡∏´‡∏ô‡∏∏‡∏Å‡∏Ñ‡∏±‡∏ö ‡πÅ‡∏ô‡∏∞‡∏ô‡∏≥ 10/10
58209,--ace--,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :) ...
58208,--ace--,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed
32142,--ionex--,730,No ratings yet,True,"it done brokeded on me, the game no longer wor..."
32143,--ionex--,105600,No ratings yet,True,"It's an amazing game, and im glad that mac use..."


Una vez exportanto todos nuestro archivos podemos concluir con la ETL de manera satisfactoriamente, ahora ya estamos listos para continiuar con las funciones. Primeramente se van a desarrollar en un jupyter notebook, posteriormente ya que todas las funciones trabajen de manera apropiada se va a pasar todo a un main.py para su ejecucion en FastApi.